### Import Libraries

In [50]:
import cv2
import numpy as np

### Load YOLO

In [68]:
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
#print('layer_names',layer_names)
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

### Load COCO classes

In [69]:
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

### Image preparing

In [70]:
conf_threshold1=0.5
distance_threshold = 10

In [71]:
image = cv2.imread("distance img.jpg") 
height, width, channels = image.shape

In [72]:
blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

### Detect the objects in the image

In [73]:
boxes = []
confidences = []
class_ids = []

In [74]:


for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        #print('confidence',confidence)
        if confidence > conf_threshold1 and class_id == 0:  # 0 is the class id for 'person'
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            #print('confidences',len(confidences))
            class_ids.append(class_id)

In [75]:
indexes = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold1, 0.4)
#indexes = cv2.dnn.NMSBoxes(boxes, confidences,conf_threshold1,nms_threshold=0)
#if len(indexes) > 0:
for i in indexes:
        x, y, w, h = boxes[i]
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
#print('indexes',indexes)
#print('boxes',boxes)
#print('indexes',len(indexes))
#print('boxes',len(boxes))

### calculate distance

In [76]:
def calculate_centroid(bbox):
  x, y, w, h = bbox
  centroid_x = int(x + (w / 2))
  centroid_y = int(y + (h / 2))
  return (centroid_x, centroid_y)

In [77]:
def get_all_centroids(boxes):
  centroids = []
  for bbox in boxes:
    centroid = calculate_centroid(bbox)  # Assuming 'calculate_centroid' is defined
    centroids.append(centroid)
  return centroids

In [78]:
def calculate_all_distances(centroids):
  distances = {}
  for i, centroid1 in enumerate(centroids):
    for j, centroid2 in enumerate(centroids):
      # Skip calculating distance for the same person (i == j)
      if i != j:
        distance = cv2.norm(np.array(centroid1), np.array(centroid2), cv2.NORM_L2)
        distances[(i, j)] = distance
  return distances

In [79]:
D=calculate_all_distances(get_all_centroids(boxes))

In [80]:
C=get_all_centroids(boxes)

In [81]:
tolerance=50
for (i, j), distance in D.items():
    if abs(C[i][0] - C[j][0]) <= tolerance and distance < distance_threshold:
      # People are close neighbors and violate social distancing
      centroid1, centroid2 = C[i], C[j]
      # Calculate midpoint between centroids
      midpoint_x = int((centroid1[0] + centroid2[0]) / 2)
      midpoint_y = int((centroid1[1] + centroid2[1]) / 2- 0.7 * 10)
      text = f"{distance:.2f}"  # Format distance to 2 decimal places
       # Adjust text position based on line direction (optional)
      if centroid1[0] < centroid2[0]:  # Line from left to right
        text_x = midpoint_x + 5  # Adjust horizontal offset
      else:
        text_x = midpoint_x - (len(text) * 0.7 * 8)  # Adjust for text width
 
      # Draw text with formatted distance and violation status
           #text += " (TOO CLOSE)"  # Violation message for close proximity
      color = (0, 0, 255)  # Red for violation
      #image_with_line = cv2.line(image, centroid1, centroid2, color, 2)
      cv2.putText(image, text, (midpoint_x, midpoint_y - 5), cv2.FONT_HERSHEY_SIMPLEX,
                  0.7, color, 2)  # Adjust font size and position as needed


In [82]:
cv2.imshow("Image with Distances", image)
cv2.waitKey(0)
cv2.destroyAllWindows()